In [2]:
# coding: utf-8
"""
・knpによる構文取得
"""
import CaboCha
import xml.etree.ElementTree as ET
from collections import Counter
import csv
import matplotlib.pyplot as plt
import copy
from pyknp import KNP

%matplotlib inline

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def select_normalization_representative_notation(fstring):
    """ 正規化代表表記を抽出します
    """
    begin = fstring.find(u'正規化代表表記:')
    end = fstring.find(u'/', begin + 1)
    return fstring[begin + len(u'正規化代表表記:') : end]

def select_dependency_structure(line):
    """係り受け構造を抽出します
    """
    # KNP
    knp = KNP(option = '-tab -anaphora')
    # 解析
    result = knp.parse(line)
    # 文節リスト
    bnst_list = result.bnst_list()
    # 文節リストをidによるディクショナリ化する
    bnst_dic = dict((x.bnst_id, x) for x in bnst_list)
    tuples = []
    for bnst in bnst_list:
        if bnst.parent_id != -1:
            # (from, to)
            tuples.append((select_normalization_representative_notation(bnst.fstring),
                           select_normalization_representative_notation(bnst_dic[bnst.parent_id].fstring)))
    return tuples

# 文字に色をつけて返す
def color(word, color='BLUE'):
    BLUE = '\033[34m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    RED = '\033[31m'
    ENDC = '\033[0m'
    if color == 'BLUE':
        return BLUE + word + ENDC
    elif color == 'GREEN':
        return GREEN + word + ENDC
    elif color == 'RED':
        return RED + word + ENDC
    elif color == 'YELLOW':
        return YELLOW + word + ENDC
    else:
        return word

In [17]:
# csvファイルから文章を読み込んで、「。」を基準にセンテンスごとに分割する
list_sentences = readcsv("./files/file_1.csv")
list_sentences = [row[1] for row in list_sentences]
list_sentences_rev = []
for row in list_sentences:
    list_sentence = row.split("。")
    list_sentences_rev.extend(list_sentence)

In [3]:
tuples = select_dependency_structure(u"日々退屈にならないように趣向が凝らされたアクティビティがあります。")

In [4]:
for row in tuples:
    print row[0], "=>", row[1]

日々 => 退屈だ
退屈だ => 凝らす
趣向 => 凝らす
凝らす => アクティビティ
アクティビティ => 有る
